In [ ]:
import numpy as np
import cv2

def rgb_to_hsi_manual(img_rgb: np.ndarray):
    """
    Converte manualmente uma imagem RGB para HSI.
    - img_rgb: H×W×3, canais em 0–255 (uint8)
    Retorna três arrays float32:
    - H: matiz em graus [0, 360)
    - S: saturação em [0, 1]
    - I: intensidade em [0, 1]
    """
    rgb = img_rgb.astype(np.float32) / 255.0
    R, G, B = rgb[...,0], rgb[...,1], rgb[...,2]

    # Intensity
    I = (R + G + B) / 3.0

    # Saturation
    min_rgb = np.minimum(np.minimum(R, G), B)
    S = np.zeros_like(I)
    mask = I > 0
    S[mask] = 1 - (min_rgb[mask] / I[mask])

    # Hue
    num = 0.5 * ((R - G) + (R - B))
    den = np.sqrt((R - G)**2 + (R - B)*(G - B)) + 1e-10
    theta = np.arccos(np.clip(num / den, -1, 1))
    H = np.degrees(theta)
    H[B > G] = 360 - H[B > G]
    H[~mask] = 0

    return H, S, I

def hsi_to_rgb_manual(H, S, I):
    """
    Converte H, S, I de volta para RGB (uint8 0–255).
    - H: matiz em graus [0,360)
    - S: saturação em [0,1]
    - I: intensidade em [0,1]
    """
    R = np.zeros_like(I)
    G = np.zeros_like(I)
    B = np.zeros_like(I)

    # Setores de H
    # 0° ≤ H < 120°
    mask1 = (H < 120)
    h1 = H[mask1]
    B[mask1] = I[mask1] * (1 - S[mask1])
    R[mask1] = I[mask1] * (1 + S[mask1] * np.cos(np.deg2rad(h1)) / np.cos(np.deg2rad(60 - h1)))
    G[mask1] = 3*I[mask1] - (R[mask1] + B[mask1])

    # 120° ≤ H < 240°
    mask2 = (H >= 120) & (H < 240)
    h2 = H[mask2] - 120
    R[mask2] = I[mask2] * (1 - S[mask2])
    G[mask2] = I[mask2] * (1 + S[mask2] * np.cos(np.deg2rad(h2)) / np.cos(np.deg2rad(60 - h2)))
    B[mask2] = 3*I[mask2] - (R[mask2] + G[mask2])

    # 240° ≤ H < 360°
    mask3 = (H >= 240)
    h3 = H[mask3] - 240
    G[mask3] = I[mask3] * (1 - S[mask3])
    B[mask3] = I[mask3] * (1 + S[mask3] * np.cos(np.deg2rad(h3)) / np.cos(np.deg2rad(60 - h3)))
    R[mask3] = 3*I[mask3] - (G[mask3] + B[mask3])

    rgb = np.stack([R, G, B], axis=-1)
    rgb = np.clip(rgb, 0, 1)
    return (rgb * 255).astype(np.uint8)

if __name__ == "__main__":
    # 1) Leitura
    bgr = cv2.imread("lymphomaNonHodgkin.jpg", cv2.IMREAD_COLOR)
    if bgr is None:
        raise FileNotFoundError("Arquivo 'lymphomaNonHodgkin.jpg' não encontrado.")
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    # 2) RGB → HSI
    H, S, I = rgb_to_hsi_manual(rgb)

    # 3) Escala H, S, I para uint8 0–255
    H_img = (H / 360.0 * 255).astype(np.uint8)
    S_img = (S * 255).astype(np.uint8)
    I_img = (I * 255).astype(np.uint8)

    # 4) Equalização do canal I
    I_eq_img = cv2.equalizeHist(I_img)

    # 5) Reconverte H, S e I equalizado para float nas faixas originais
    H_f = H_img.astype(np.float32) / 255.0 * 360.0
    S_f = S_img.astype(np.float32) / 255.0
    I_f_eq = I_eq_img.astype(np.float32) / 255.0

    # 6) HSI → RGB equalizado
    rgb_eq = hsi_to_rgb_manual(H_f, S_f, I_f_eq)
    bgr_eq = cv2.cvtColor(rgb_eq, cv2.COLOR_RGB2BGR)

    # 7) Concatena lado a lado e exibe
    concat = np.hstack((bgr, bgr_eq))
    cv2.imshow("Original (esq) vs Equalizada (dir)", concat)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
